# Sonification of temporal triggered sound events
In numerous MIR applications one encounters cases where - for the time being not further specified - events occur at certain temporal positions. 
In this notebook, we illustrate how the **libsoni.tse** module can be used to sonify these events with respect to their characteritics.

In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
from IPython import display as ipd

from libsoni.core import tse
from libsoni.util.utils import mix_sonification_and_original

Fs= 22050

## Simple scenario: time positions
To start with a simple example, let's create a list of arbitrarily chosen time positions, given in seconds:

In [ ]:
# Define arbitrarily chosen time positions
arbitrarily_chosen_time_positions = [0.5, 1.25, 2.5, 2.75]

Assuming we want to sonify this list, we can use the **sonify_tse_click** function to generate an audio signal comprising clicks at the corresponding time positions.

### Sonification of time positions with clicks

In [ ]:
# Sonification using libsoni
sonified_arbitrarily_chosen_time_positions = tse.sonify_tse_click(
                                             time_positions=arbitrarily_chosen_time_positions)

print('Sonified time positions:')
ipd.display(ipd.Audio(sonified_arbitrarily_chosen_time_positions, rate=Fs))

### Customizing the Sonification
The clicks generated within **sonify_tse_click** can be adjusted in order to match the respective use case with the following parameters:
 - **click_pitch**: pitch of the click
 - **click_duration**: duration of the click
 - **click_amplitude**: amplitude of the click
 
Let's say we'd rather have deeper sounding, longer clicks, the above sonification changes as follows:

In [ ]:
# Sonification using libsoni
new_sonified_arbitrarily_chosen_time_positions = tse.sonify_tse_click(
                                                 time_positions=arbitrarily_chosen_time_positions,
                                                 click_pitch=43,
                                                 click_duration = 0.5)

print('Sonified time positions with deeper, longer clicks:')
ipd.display(ipd.Audio(new_sonified_arbitrarily_chosen_time_positions, rate=Fs))

### Sonification of time positions with samples
Before we get to a real-world audio example, let's stay with the arbitrary time positions and explain the way in which samples can be used instead of clicks.
The first thing we need for this is - guess three times - a sample. Why don't we use a finger snap sound?

In [ ]:
# Load sample
snap_sample, _ = librosa.load(os.path.join('data_audio','samples', 'snap.wav'), sr=Fs)

print('Snap sample:')
ipd.display(ipd.Audio(snap_sample, rate=Fs))

Now, using the function **sonify_tse_sample**, we can sonify our list of time positions with our sample:

In [ ]:
# Sonification using libsoni
sample_sonified_arbitrarily_chosen_time_positions = tse.sonify_tse_sample(
                                                 time_positions=arbitrarily_chosen_time_positions,
                                                 sample=snap_sample)
                                                 
print('Sonified time positions with snap sample:')
ipd.display(ipd.Audio(sample_sonified_arbitrarily_chosen_time_positions, rate=Fs))

## Scenario 1: Measure Annotations

## Scenario 2: Beat Annotations

## Scenario 3: Drumset